In [ ]:
pip install elasticsearch -qq 

In [ ]:
!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-8.14.0-linux-x86_64.tar.gz
!tar -xzf elasticsearch-8.14.0-linux-x86_64.tar.gz
!chown -R root:root elasticsearch-8.14.0
!adduser --disabled-password --gecos '' elasticsearch_user
!chown -R elasticsearch_user:elasticsearch_user elasticsearch-8.14.0

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import io

import requests
from tqdm.notebook import tqdm
import subprocess
import time
import json
import elasticsearch
from elasticsearch import Elasticsearch

In [7]:
# Run elasticsearch
command = "sudo -u elasticsearch_user ./elasticsearch-8.14.0/bin/elasticsearch -E xpack.security.enabled=false -E thread_pool.search.size=1 -E thread_pool.write.size=1"
process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
time.sleep(20)

In [8]:
%%bash
# If you get 1 root & 2 daemon process then Elasticsearch instance has started successfully
ps -ef | grep elasticsearch

root         562     520  0 15:24 ?        00:00:00 /bin/sh -c sudo -u elasticsearch_user ./elasticsearch-8.14.0/bin/elasticsearch -E xpack.security.enabled=false -E thread_pool.search.size=1 -E thread_pool.write.size=1
root         563     562  0 15:24 ?        00:00:00 sudo -u elasticsearch_user ./elasticsearch-8.14.0/bin/elasticsearch -E xpack.security.enabled=false -E thread_pool.search.size=1 -E thread_pool.write.size=1
elastic+     564     563 28 15:24 ?        00:00:06 /kaggle/working/elasticsearch-8.14.0/jdk/bin/java -Xms4m -Xmx64m -XX:+UseSerialGC -Dcli.name=server -Dcli.script=./elasticsearch-8.14.0/bin/elasticsearch -Dcli.libs=lib/tools/server-cli -Des.path.home=/kaggle/working/elasticsearch-8.14.0 -Des.path.conf=/kaggle/working/elasticsearch-8.14.0/config -Des.distribution.type=tar -cp /kaggle/working/elasticsearch-8.14.0/lib/*:/kaggle/working/elasticsearch-8.14.0/lib/cli-launcher/* org.elasticsearch.launcher.CliToolLauncher -E xpack.security.enabled=false -E thread_pool.se

In [13]:
es_client = Elasticsearch('http://localhost:9200/', request_timeout=60) 

In [15]:
!curl localhost:9200

{
  "name" : "a8cc6a5b6862",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "vpUsZrWEQkCRzFnZIxspMQ",
  "version" : {
    "number" : "8.14.0",
    "build_flavor" : "default",
    "build_type" : "tar",
    "build_hash" : "8d96bbe3bf5fed931f3119733895458eab75dca9",
    "build_date" : "2024-06-03T10:05:49.073003402Z",
    "build_snapshot" : false,
    "lucene_version" : "9.10.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [17]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [18]:
documents[1]

{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp'}

In [79]:
set([i['course'] for  i in documents])
    
    

{'data-engineering-zoomcamp', 'machine-learning-zoomcamp', 'mlops-zoomcamp'}

In [ ]:
pip install sentence_transformers==2.7.0

In [137]:
from sentence_transformers import SentenceTransformer

# if you get an error do the following:
# 1. Uninstall numpy 
# 2. Uninstall torch
# 3. pip install numpy==1.26.4
# 4. pip install torch
# run the above cell, it should work
model = SentenceTransformer("multi-qa-mpnet-base-cos-v1",)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.25k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [138]:
len(model.encode("This is a simple sentence",show_progress_bar=False))

768

In [21]:
import os

# Set the environment variable
os.environ["TOKENIZERS_PARALLELISM"] = "true" 

In [191]:
#created the dense vector using the pre-trained model
operations = []
for doc in tqdm(documents):
    # Transforming the title into an embedding using the model
    doc["text_vector"] = model.encode(doc["text"],show_progress_bar=False,normalize_embeddings=True).tolist()
    operations.append(doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [178]:
operations[1]-

SyntaxError: invalid syntax (2274040940.py, line 1)

In [24]:
print(es_client.info())

{'name': 'a8cc6a5b6862', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'vpUsZrWEQkCRzFnZIxspMQ', 'version': {'number': '8.14.0', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': '8d96bbe3bf5fed931f3119733895458eab75dca9', 'build_date': '2024-06-03T10:05:49.073003402Z', 'build_snapshot': False, 'lucene_version': '9.10.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [193]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} ,
            "text_vector":{"type":"dense_vector","dims": 768,"index":True,"similarity": "cosine"
        },
        }
    }
}

In [194]:
index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [127]:
!curl -X GET "http://localhost:9200"


{
  "name" : "a8cc6a5b6862",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "vpUsZrWEQkCRzFnZIxspMQ",
  "version" : {
    "number" : "8.14.0",
    "build_flavor" : "default",
    "build_type" : "tar",
    "build_hash" : "8d96bbe3bf5fed931f3119733895458eab75dca9",
    "build_date" : "2024-06-03T10:05:49.073003402Z",
    "build_snapshot" : false,
    "lucene_version" : "9.10.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [195]:
for doc in operations:
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

In [196]:
search_term = "Windows or mac?"
vector_search_term = model.encode(search_term)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [184]:
vector_search_term-

SyntaxError: invalid syntax (200590494.py, line 1)

In [198]:
query = {
    "field" : "text_vector",
    "query_vector" :  vector_search_term,
    "k" : 5,
    "num_candidates" : 10000, 
}

In [199]:
res = es_client.search(index=index_name, knn=query,source=["text","section","question","course"])

In [200]:
res["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': 'WF-BdJABmYPefPWXFkCi',
  '_score': 0.7180941,
  '_source': {'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully',
   'section': 'General course-related questions',
   'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': 'jV-BdJABmYPefPWXM0Kz',
  '_score': 0.648803,
  '_source': {'text': 'Refer to the page https://docs.docker.com/desktop/install/mac-install/ remember to check if you have apple chip or intel chip.',
   'section': '5. Deploying Machine Learning Models',
   'question': 'Install docker on MacOS',
   'course': 'machine-learning-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': 'LF-BdJABmYPefPWXLkLn',
  '_score': 0.62542486,
  '_source': {'text': 'That’s normal. We all have different environments: our computers have different versions of OS and different version

In [201]:
response = es_client.search(
    index=index_name,
    size=5,
    query={
        "bool": {
            "must": {
             "multi_match": 
                        {"query": "windows or python?", 
                         "fields": ["text", "question","course","title"],
                         "type": "best_fields"
                        }
                    },
            "filter": {
                "term": {
                        "course": "data-engineering-zoomcamp"
            }
        }
        }
    }
)

In [204]:
response["hits"]["hits"][1]-

SyntaxError: invalid syntax (2981941790.py, line 1)